In [1]:
import warnings
warnings.filterwarnings("ignore")

import ast
import os
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import torch
import importlib
import cv2 

from shutil import copyfile
from tqdm.notebook import tqdm
tqdm.pandas()
from sklearn.model_selection import GroupKFold
from PIL import Image
from string import Template
from IPython.display import display

TRAIN_PATH = 'C:\\Users\\yangd\\Documents\\Python_Scripts\\TensorFlow\\tensorflow-great-barrier-reef'

import codecs, sys





In [2]:
# check Torch and CUDA version
print(f"Torch: {torch.__version__}")
!nvcc --version

Torch: 1.10.0+cpu
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:35_Pacific_Daylight_Time_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.relgpu_drvr445TC445_37.28845127_0


In [3]:
!git clone https://github.com/Megvii-BaseDetection/YOLOX -q

%cd YOLOX
!pip install -U pip && pip install -r requirements.txt
!pip install -v -e . 

C:\Users\yangd\Documents\Python_Scripts\TensorFlow\YOLOX


fatal: destination path 'YOLOX' already exists and is not an empty directory.


Using pip 21.2.2 from C:\Users\yangd\Anaconda3\lib\site-packages\pip (python 3.8)
Obtaining file:///C:/Users/yangd/Documents/Python_Scripts/TensorFlow/YOLOX
  Attempting uninstall: yolox
    Found existing installation: yolox 0.1.0
    Uninstalling yolox-0.1.0:
      Removing file or directory c:\users\yangd\anaconda3\lib\site-packages\yolox.egg-link
      Removing pth entries from c:\users\yangd\anaconda3\lib\site-packages\easy-install.pth:
      Removing entry: c:\users\yangd\documents\python_scripts\tensorflow\yolox
      Successfully uninstalled yolox-0.1.0
  Running setup.py develop for yolox


    Running command python setup.py egg_info
    No CUDA runtime is found, using CUDA_HOME='C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.0'
    running egg_info
    creating C:\Users\yangd\AppData\Local\Temp\pip-pip-egg-info-zin089p0\yolox.egg-info
    writing C:\Users\yangd\AppData\Local\Temp\pip-pip-egg-info-zin089p0\yolox.egg-info\PKG-INFO
    writing dependency_links to C:\Users\yangd\AppData\Local\Temp\pip-pip-egg-info-zin089p0\yolox.egg-info\dependency_links.txt
    writing top-level names to C:\Users\yangd\AppData\Local\Temp\pip-pip-egg-info-zin089p0\yolox.egg-info\top_level.txt
    writing manifest file 'C:\Users\yangd\AppData\Local\Temp\pip-pip-egg-info-zin089p0\yolox.egg-info\SOURCES.txt'
    reading manifest file 'C:\Users\yangd\AppData\Local\Temp\pip-pip-egg-info-zin089p0\yolox.egg-info\SOURCES.txt'
    writing manifest file 'C:\Users\yangd\AppData\Local\Temp\pip-pip-egg-info-zin089p0\yolox.egg-info\SOURCES.txt'
    Error in atexit._run_exitfuncs:
    Traceback (m

In [4]:
def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_path(row):
    row['image_path'] = f'{TRAIN_PATH}/train_images/video_{row.video_id}/{row.video_frame}.jpg'
    return row

In [5]:
df = pd.read_csv("C:\\Users\\yangd\\Documents\\Python_Scripts\\TensorFlow\\tensorflow-great-barrier-reef\\train.csv")
df.head(5)

,video_id,sequence,video_frame,sequence_frame,image_id,annotations
0,0,40258,0,0,0-0,[]
1,0,40258,1,1,0-1,[]
2,0,40258,2,2,0-2,[]
3,0,40258,3,3,0-3,[]
4,0,40258,4,4,0-4,[]


In [6]:
# Taken only annotated photos
df["num_bbox"] = df['annotations'].apply(lambda x: str.count(x, 'x'))
df_train = df[df["num_bbox"]>0]

#Annotations 
df_train['annotations'] = df_train['annotations'].progress_apply(lambda x: ast.literal_eval(x))
df_train['bboxes'] = df_train.annotations.progress_apply(get_bbox)

#Images resolution
df_train["width"] = 1280
df_train["height"] = 720

#Path of images
df_train = df_train.progress_apply(get_path, axis=1)

  0%|          | 0/4919 [00:00<?, ?it/s]

  0%|          | 0/4919 [00:00<?, ?it/s]

  0%|          | 0/4919 [00:00<?, ?it/s]

In [7]:
kf = GroupKFold(n_splits = 5) 
df_train = df_train.reset_index(drop=True)
df_train['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(kf.split(df_train, y = df_train.video_id.tolist(), groups=df_train.sequence)):
    df_train.loc[val_idx, 'fold'] = fold

df_train.head(5)

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,num_bbox,bboxes,width,height,image_path,fold
0,0,40258,16,16,0-16,"[{'x': 559, 'y': 213, 'width': 50, 'height': 32}]",1,"[[559, 213, 50, 32]]",1280,720,C:\Users\yangd\Documents\Python_Scripts\Tensor...,4
1,0,40258,17,17,0-17,"[{'x': 558, 'y': 213, 'width': 50, 'height': 32}]",1,"[[558, 213, 50, 32]]",1280,720,C:\Users\yangd\Documents\Python_Scripts\Tensor...,4
2,0,40258,18,18,0-18,"[{'x': 557, 'y': 213, 'width': 50, 'height': 32}]",1,"[[557, 213, 50, 32]]",1280,720,C:\Users\yangd\Documents\Python_Scripts\Tensor...,4
3,0,40258,19,19,0-19,"[{'x': 556, 'y': 214, 'width': 50, 'height': 32}]",1,"[[556, 214, 50, 32]]",1280,720,C:\Users\yangd\Documents\Python_Scripts\Tensor...,4
4,0,40258,20,20,0-20,"[{'x': 555, 'y': 214, 'width': 50, 'height': 32}]",1,"[[555, 214, 50, 32]]",1280,720,C:\Users\yangd\Documents\Python_Scripts\Tensor...,4


In [8]:
HOME_DIR = 'C:\\Users\\yangd\\Documents\\Python_Scripts\\TensorFlow\\' 
DATASET_PATH = 'dataset\\images'

!mkdir {HOME_DIR}dataset
!mkdir {HOME_DIR}{DATASET_PATH}
!mkdir {HOME_DIR}{DATASET_PATH}\\train2017
!mkdir {HOME_DIR}{DATASET_PATH}\\val2017
!mkdir {HOME_DIR}{DATASET_PATH}\\annotations

A subdirectory or file C:\Users\yangd\Documents\Python_Scripts\TensorFlow\dataset already exists.
A subdirectory or file C:\Users\yangd\Documents\Python_Scripts\TensorFlow\dataset\images already exists.
A subdirectory or file C:\Users\yangd\Documents\Python_Scripts\TensorFlow\dataset\images\\train2017 already exists.
A subdirectory or file C:\Users\yangd\Documents\Python_Scripts\TensorFlow\dataset\images\\val2017 already exists.
A subdirectory or file C:\Users\yangd\Documents\Python_Scripts\TensorFlow\dataset\images\\annotations already exists.


In [9]:
SELECTED_FOLD = 4

for i in tqdm(range(len(df_train))):
    row = df_train.loc[i]
    if row.fold != SELECTED_FOLD:
        copyfile(f'{row.image_path}', f'{HOME_DIR}{DATASET_PATH}/train2017/{row.image_id}.jpg')
    else:
        copyfile(f'{row.image_path}', f'{HOME_DIR}{DATASET_PATH}/val2017/{row.image_id}.jpg') 

  0%|          | 0/4919 [00:00<?, ?it/s]

In [10]:
print(f'Number of training files: {len(os.listdir(f"{HOME_DIR}{DATASET_PATH}/train2017/"))}')
print(f'Number of validation files: {len(os.listdir(f"{HOME_DIR}{DATASET_PATH}/val2017/"))}')

Number of training files: 3974
Number of validation files: 945


In [11]:
def save_annot_json(json_annotation, filename):
    with open(filename, 'w') as f:
        output_json = json.dumps(json_annotation)
        f.write(output_json)

In [12]:
annotion_id = 0

In [13]:
def dataset2coco(df, dest_path):
    
    global annotion_id
    
    annotations_json = {
        "info": [],
        "licenses": [],
        "categories": [],
        "images": [],
        "annotations": []
    }
    
    info = {
        "year": "2021",
        "version": "1",
        "description": "COTS dataset - COCO format",
        "contributor": "",
        "url": "https://kaggle.com",
        "date_created": "2021-11-30T15:01:26+00:00"
    }
    annotations_json["info"].append(info)
    
    lic = {
            "id": 1,
            "url": "",
            "name": "Unknown"
        }
    annotations_json["licenses"].append(lic)

    classes = {"id": 0, "name": "starfish", "supercategory": "none"}

    annotations_json["categories"].append(classes)

    
    for ann_row in df.itertuples():
            
        images = {
            "id": ann_row[0],
            "license": 1,
            "file_name": ann_row.image_id + '.jpg',
            "height": ann_row.height,
            "width": ann_row.width,
            "date_captured": "2021-11-30T15:01:26+00:00"
        }
        
        annotations_json["images"].append(images)
        
        bbox_list = ann_row.bboxes
        
        for bbox in bbox_list:
            b_width = bbox[2]
            b_height = bbox[3]
            
            # some boxes in COTS are outside the image height and width
            if (bbox[0] + bbox[2] > 1280):
                b_width = bbox[0] - 1280 
            if (bbox[1] + bbox[3] > 720):
                b_height = bbox[1] - 720 
                
            image_annotations = {
                "id": annotion_id,
                "image_id": ann_row[0],
                "category_id": 0,
                "bbox": [bbox[0], bbox[1], b_width, b_height],
                "area": bbox[2] * bbox[3],
                "segmentation": [],
                "iscrowd": 0
            }
            
            annotion_id += 1
            annotations_json["annotations"].append(image_annotations)
        
        
    print(f"Dataset COTS annotation to COCO json format completed! Files: {len(df)}")
    return annotations_json

In [14]:
# Convert COTS dataset to JSON COCO
train_annot_json = dataset2coco(df_train[df_train.fold != SELECTED_FOLD], f"{HOME_DIR}{DATASET_PATH}/train2017/")
val_annot_json = dataset2coco(df_train[df_train.fold == SELECTED_FOLD], f"{HOME_DIR}{DATASET_PATH}/val2017/")

# Save converted annotations
save_annot_json(train_annot_json, f"{HOME_DIR}{DATASET_PATH}/annotations/train.json")
save_annot_json(val_annot_json, f"{HOME_DIR}{DATASET_PATH}/annotations/valid.json")

Dataset COTS annotation to COCO json format completed! Files: 3974
Dataset COTS annotation to COCO json format completed! Files: 945


In [15]:
# Choose model for your experiments NANO or YOLOX-S (you can adapt for other model type)

NANO = False

In [16]:
config_file_template = '''

#!/usr/bin/env python3
# -*- coding:utf-8 -*-
# Copyright (c) Megvii, Inc. and its affiliates.

import os

from yolox.exp import Exp as MyExp


class Exp(MyExp):
    def __init__(self):
        super(Exp, self).__init__()
        self.depth = 0.33
        self.width = 0.50
        self.exp_name = os.path.split(os.path.realpath(__file__))[1].split(".")[0]
        
        # Define yourself dataset path
        self.data_dir = "/kaggle/working/dataset/images"
        self.train_ann = "train.json"
        self.val_ann = "valid.json"

        self.num_classes = 1

        self.max_epoch = $max_epoch
        self.data_num_workers = 2
        self.eval_interval = 1
        
        self.mosaic_prob = 1.0
        self.mixup_prob = 1.0
        self.hsv_prob = 1.0
        self.flip_prob = 0.5
        self.no_aug_epochs = 2
        
        self.input_size = (960, 960)
        self.mosaic_scale = (0.5, 1.5)
        self.random_size = (10, 20)
        self.test_size = (960, 960)
'''

In [17]:
if NANO:
    config_file_template = '''

#!/usr/bin/env python3
# -*- coding:utf-8 -*-
# Copyright (c) Megvii, Inc. and its affiliates.

import os

import torch.nn as nn

from yolox.exp import Exp as MyExp


class Exp(MyExp):
    def __init__(self):
        super(Exp, self).__init__()
        self.depth = 0.33
        self.width = 0.25
        self.input_size = (416, 416)
        self.mosaic_scale = (0.5, 1.5)
        self.random_size = (10, 20)
        self.test_size = (416, 416)
        self.exp_name = os.path.split(
            os.path.realpath(__file__))[1].split(".")[0]
        self.enable_mixup = False

        # Define yourself dataset path
        self.data_dir = "/kaggle/working/dataset/images"
        self.train_ann = "train.json"
        self.val_ann = "valid.json"

        self.num_classes = 1

        self.max_epoch = $max_epoch
        self.data_num_workers = 2
        self.eval_interval = 1

    def get_model(self, sublinear=False):
        def init_yolo(M):
            for m in M.modules():
                if isinstance(m, nn.BatchNorm2d):
                    m.eps = 1e-3
                    m.momentum = 0.03

        if "model" not in self.__dict__:
            from yolox.models import YOLOX, YOLOPAFPN, YOLOXHead
            in_channels = [256, 512, 1024]
            # NANO model use depthwise = True, which is main difference.
            backbone = YOLOPAFPN(self.depth,
                                 self.width,
                                 in_channels=in_channels,
                                 depthwise=True)
            head = YOLOXHead(self.num_classes,
                             self.width,
                             in_channels=in_channels,
                             depthwise=True)
            self.model = YOLOX(backbone, head)

        self.model.apply(init_yolo)
        self.model.head.initialize_biases(1e-2)
        return self.model

'''

In [18]:
PIPELINE_CONFIG_PATH='cots_config.py'

pipeline = Template(config_file_template).substitute(max_epoch = 20)

with open(PIPELINE_CONFIG_PATH, 'w') as f:
    f.write(pipeline)

In [19]:
# ./yolox/data/datasets/voc_classes.py

voc_cls = '''
VOC_CLASSES = (
  "starfish",
)
'''
with open('.\\yolox\\data\\datasets\\voc_classes.py', 'w') as f:
    f.write(voc_cls)

# ./yolox/data/datasets/coco_classes.py

coco_cls = '''
COCO_CLASSES = (
  "starfish",
)
'''
with open('.\\yolox\\data\\datasets\\coco_classes.py', 'w') as f:
    f.write(coco_cls)



In [21]:
# check if everything is ok    
%more .\\yolox\\data\\datasets\\coco_classes.py

In [22]:

sh = 'wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_s.pth'
MODEL_FILE = 'yolox_s.pth'

if NANO:
    sh = '''
    wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_nano.pth
    '''
    MODEL_FILE = 'yolox_nano.pth'

with open('script.sh', 'w') as file:
  file.write(sh)



In [23]:
!cp ./tools/train.py ./

In [24]:
from platform import python_version

print(python_version())

3.8.8


In [25]:
!python train.py \
    -f cots_config.py \
    -d 1 \
    -b 32 \
    --fp16 \
    -o \
    -c {MODEL_FILE}   # Remember to change this line if you take different model eg. yolo_nano.pth, yolox_s.pth or yolox_m.pth

--- Logging error in Loguru Handler #2 ---
--- Logging error in Loguru Handler #2 ---


C:\Users\yangd\Anaconda3\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
2021-12-17 23:15:19 | INFO     | yolox.core.trainer:126 - args: Namespace(batch_size=32, cache=False, ckpt='yolox_s.pth', devices=1, dist_backend='nccl', dist_url=None, exp_file='cots_config.py', experiment_name='cots_config', fp16=True, machine_rank=0, name=None, num_machines=1, occupy=True, opts=['#', 'Remember', 'to', 'change', 'this', 'line', 'if', 'you', 'take', 'different', 'model', 'eg.', 'yolo_nano.pth,', 'yolox_s.pth', 'or', 'yolox_m.pth'], resume=False, start_epoch=None)
2021-12-17 23:15:19 | INFO     | yolox.core.trainer:127 - exp value:
\u2552\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2564\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\